<a href="https://colab.research.google.com/github/KiselevaStacy/python-ai--/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о породах собак

**Цель**: Научиться читать CSV-файлы из вашего репозитория GitHub в Google Colab и выполнять базовую проверку данных о породах собак с помощью pandas.

**Данные:**
- `dog_breeds.csv` — информация о породах собак (страна происхождения, рост, масса, продолжительность жизни)

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл с данными о собаках в pandas DataFrame
3. Очищаем и анализируем структуру данных
4. Проверяем основные статистики (рост, масса, продолжительность жизни)

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🐶 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai--"):
    !git clone -q https://github.com/KiselevaStacy/python-ai--.git

%cd python-ai--

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai--")

/content/python-ai--
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai--


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [10]:
# 🐶 Шаг 2. Чтение данных о породах собак в pandas

import pandas as pd

df_dogs = pd.read_csv("data/dog_breeds.csv")

print("✅ Загружено строк в df_dogs:", len(df_dogs))
print("\n📋 Первые 3 строки данных:")
print(df_dogs.head(3))

✅ Загружено строк в df_dogs: 188

📋 Первые 3 строки данных:
                               dogBreed dogBreedLabel countryLabel  height  \
0  http://www.wikidata.org/entity/Q5414  Немецкий дог     Германия    72.0   
1  http://www.wikidata.org/entity/Q5414  Немецкий дог     Германия    80.0   
2  http://www.wikidata.org/entity/Q5414  Немецкий дог     Германия    84.0   

   mass  lifeExpectancy  
0  90.0             9.0  
1  90.0             9.0  
2  90.0             9.0  


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `dogBreed` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно названия породы.
- Столбцы `dogBreedLabel`, `countryLabel` содержат читаемые подписи (название породы, страна происхождения).

В этом шаге мы:
- удалим столбец с URL Wikidata (`dogBreed`);
- переименуем `dogBreedLabel → dogBreed`, `countryLabel → country`;
- приведём числовые столбцы (`height`, `mass`, `lifeExpectancy`) к числовому типу.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0 (для `height` и `mass`);
- `fillna(df["lifeExpectancy"].median())` — для продолжительности жизни логичнее заполнить медианой, а не нулём.

> ⚠️ **Важно:** удаление технических столбцов и переименование `*Label` делает таблицу удобной для дальнейшего анализа пород собак.

In [11]:
# 🧹 Шаг 2B. Очистка и переименование столбцов (идемпотентная версия)

# Проверяем, есть ли ещё технический столбец dogBreed с URL
if "dogBreed" in df_dogs.columns and "dogBreedLabel" in df_dogs.columns:
    # Удаляем URL-столбец ТОЛЬКО если существует отдельный столбец с названием породы
    df_dogs = df_dogs.drop(columns=["dogBreed"])
    print("✅ Удалён технический столбец dogBreed (URL Wikidata)")
else:
    print("ℹ️ Столбец dogBreed (URL) уже удалён ранее")

# Переименовываем столбцы Label → чистые названия (только если столбцы Label ещё существуют)
rename_map = {}
if "dogBreedLabel" in df_dogs.columns:
    rename_map["dogBreedLabel"] = "dogBreed"
if "countryLabel" in df_dogs.columns:
    rename_map["countryLabel"] = "country"

if rename_map:
    df_dogs = df_dogs.rename(columns=rename_map)
    print(f"✅ Переименованы столбцы: {rename_map}")
else:
    print("ℹ️ Столбцы с постфиксом Label уже переименованы ранее")

# Приводим числовые столбцы к числовому типу (безопасно для повторного запуска)
df_dogs["height"] = pd.to_numeric(df_dogs["height"], errors="coerce").fillna(0).astype(int)
df_dogs["mass"] = pd.to_numeric(df_dogs["mass"], errors="coerce").fillna(0).astype(int)

# Для продолжительности жизни используем медиану вместо 0
median_life = df_dogs["lifeExpectancy"].median()
df_dogs["lifeExpectancy"] = pd.to_numeric(df_dogs["lifeExpectancy"], errors="coerce").fillna(median_life).round(1)

print("\n✅ Данные очищены и готовы к анализу")
print("\n📋 Текущая структура таблицы:")
print(df_dogs.dtypes)
print("\n👀 Пример данных после очистки:")
print(df_dogs.head(3)[["dogBreed", "country", "height", "mass", "lifeExpectancy"]])

✅ Удалён технический столбец dogBreed (URL Wikidata)
✅ Переименованы столбцы: {'dogBreedLabel': 'dogBreed', 'countryLabel': 'country'}

✅ Данные очищены и готовы к анализу

📋 Текущая структура таблицы:
dogBreed           object
country            object
height              int64
mass                int64
lifeExpectancy    float64
dtype: object

👀 Пример данных после очистки:
       dogBreed   country  height  mass  lifeExpectancy
0  Немецкий дог  Германия      72    90             9.0
1  Немецкий дог  Германия      80    90             9.0
2  Немецкий дог  Германия      84    90             9.0


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор обоих DataFrame:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по бюджету (`capital_cost`).

Для удобства напишем маленькую функцию `show_info(df, name)`, чтобы не повторять один и тот же код два раза.

In [ ]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_cost, "Бюджеты мультфильмов (df_cost)")
show_info(df_genre, "Жанры, страны и продолжительность (df_genre)")

print("\n📈 Статистика по бюджету (capital_cost):")
print(df_cost["capital_cost"].describe())

In [ ]:
# То же самое, но в миллионах долларов (млн $)
stats_millions = (df_cost["capital_cost"] / 1e6).describe().round(2)
print("\n📈 Статистика по бюджету в млн $:")
print(stats_millions)

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных** фильмов, стран и жанров есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу строк);
- **какие жанры самые популярные** (Топ‑10 по числу строк).

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.head()` берёт первые N строк, поэтому  
`df_genre["country"].value_counts().head()` даёт **Топ‑5 стран по числу записей**.

In [ ]:
# ✅ Шаг 4. Быстрая проверка и валидация данных

print("🔍 Быстрая проверка данных")

# Датасет 1: бюджеты
print("\nУникальных мультфильмов в df_cost:", df_cost["film"].nunique())
print("Диапазон бюджетов (млн $):",
      df_cost["capital_cost"].min() / 1e6, "—", df_cost["capital_cost"].max() / 1e6)

# Датасет 2: жанры, страны, длительность
print("\nУникальных мультфильмов в df_genre:", df_genre["film"].nunique())
print("Уникальных стран:", df_genre["country"].nunique())
print("Уникальных жанров:", df_genre["genre"].nunique())

print("\nТоп-5 стран по числу записей:")
print(df_genre["country"].value_counts().head())

print("\nТоп-10 жанров:")
print(df_genre["genre"].value_counts().head(10))

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨